In [1]:
import os
import sys
import tempfile
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as opim
import torch.multiprocessing as mp

/home/nfs/admin0/lewisliu/.conda/envs/transformer/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.optim as optim

In [4]:
def example(rank, world_size):
    #create default process group
    dist.init_process_group("gloo", rank=rank, world_size=world_size)
    #create model
    model = nn.Linear(10, 10).to(rank)
    # construct DDP model
    ddp_model = DDP(model, device_ids=[rank])
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr = 0.001)
    
    outputs = ddp_model(torch.randn(20, 10).to(rank))
    labels = torch.randn(20, 10).to(rank)
    loss_fn(outputs, labels).backward()
    optimizer.step()

In [5]:
def main():
    world_size = 2
    mp.spawn(example, 
                            args=(world_size,),
                            nprocs=world_size,
                            join=True)

In [7]:
if __name__ == '__main__':
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"
    main()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/nfs/admin0/lewisliu/.conda/envs/transformer/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/nfs/admin0/lewisliu/.conda/envs/transformer/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'example' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/nfs/admin0/lewisliu/.conda/envs/transformer/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/nfs/admin0/lewisliu/.conda/envs/transformer/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'example' on <module '__main__' (built-in)>


Exception: process 1 terminated with exit code 1